In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os.path
from PIL import Image
import itertools
import scipy.io as sio

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tf.keras
from tf.keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv1D
from tf.keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tf.keras.models import Model
from tf.keras.optimizers import SGD
from tf.keras.callbacks import ModelCheckpoint,LearningRateScheduler
from tf.keras import backend as K

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
train_data = np.load('/users/jhlee/data/img_snd/train_data.npz')
test_data = np.load('/users/jhlee/data/img_snd/test_data.npz')

In [ ]:
X_train = train_data['X_train']
y_train = train_data['y_train']
X_test = test_data['X_test']
y_test = test_data['y_test']

In [ ]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_test_centered = (X_test - mean_vals)/std_val

del X_train, X_test

In [ ]:
img_dim = (99, 257)
img_dim_crop = (99, 257)

In [ ]:
def batch_generator(X, y, batch_size=64, 
                    shuffle=False, random_seed=None):
    
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
    
    for i in range(0, X.shape[0], batch_size):
        yield (X[i:i+batch_size, :], y[i:i+batch_size])

In [ ]:
def build_cnn_1conv_1fc():
    sequence_input = Input((1,159))    # also tried (159,1)
    
    conv1 = Conv1D(kernel_size=8, filters=25, strides=1, activation='relu', name='layer_conv1', padding='valid')(sequence_input)
    conv1_pool = MaxPooling1D(pool_size=4, strides=4, padding='same')(conv1)
    
    flat = Flatten()(conv1_pool)
    fc1 = Dense(1024, activation='relu', name='layer_fc1', padding='same')(flat)
    fc1_dropout = Dropout(0.5)(fc1)
    
    output_layer = fc_layer(fc1_drop, name='output_layer',
                              n_output_units=16,
                              activation_fn=None)
    
    model = Model(inputs = sequence_input, outputs = output_layer, name='predict')
    model.summary()

In [ ]:
## Loss Function and Optimization
adam_with_params = keras.optimizers.Adam(lr=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=adam_with_params,
              metrics=['accuracy'])

In [ ]:
model_path = '/users/lww/data/checkpoint/AlexNet_based_check_point/'
os.makedirs(model_path, exist_ok=True)
model_filename = model_path+'{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath = model_filename, monitor = "val_loss", 
                               verbose=1, save_best_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=100)